# The Price is Right
So this notebook is like the “next level” — now that I have a solid pricing model, I’m building a Scanning Agent that automatically looks for good product deals from RSS feeds and filters out the best ones

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [04:09<00:00, 49.81s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: Popeyes Big Box Deal for $6\nDetails: For $6 you get a 2-piece chicken signatures or 3-piece tenders, two regular sides, and a biscuit.If you're in the mood for fast food, you can save even more with these\xa0Uber Eats promo codes. Buy Now at Popeyes\nFeatures: \nURL: https://www.dealnews.com/Popeyes-Big-Box-Deal-for-6/21776210.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Hisense U7 Series 85U7K 85" 4K 144Hz ULED UHD Smart TV (2023) for $998 + free shipping
Details: It's $500 under our mention from a year ago, $700 off original list, the best price we've seen, and a solid deal on an 85" TV with these specs. Buy Now at Walmart
Features: 3840x2160 (4K) resolution 144Hz Google TV compatible with Amazon Alexa and Google Assistant HDMI 2.

In [9]:
# This is the main function that talks to OpenAI’s API.
# It sends the system_prompt and user_prompt together, asks for JSON-only output, and parses it into the DealSelection data structure
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description='The Sumry 4,000W Hybrid Solar Inverter is designed to convert DC power from solar panels to AC power for your home. It features a DC input of 34V to output AC at 110V, making it versatile for various solutions. The built-in 140A MPPT solar charger enables efficient energy management, compatible with both 24V lead-acid and lithium batteries. Its robust design ensures reliability and performance in solar energy applications, making it an excellent investment for sustainable energy use.', price=244.0, url='https://www.dealnews.com/products/Sumry/Sumry-4-000-W-Hybrid-Solar-Inverter/490788.html?iref=rss-c142')

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
# Here I’m bringing in my custom ScannerAgent, which wraps all this logic — it scrapes, runs GPT, and returns best deals automatically
agent = ScannerAgent()
result = agent.scan()

In [15]:
result

DealSelection(deals=[Deal(product_description='The Hisense U7 Series 85U7K is a massive 85" Smart TV that delivers stunning 4K resolution at 3840x2160 pixels. It boasts a fast 144Hz refresh rate, making it perfect for high-speed gaming and action-packed movies. This TV is Google TV compatible and integrates smoothly with Amazon Alexa and Google Assistant for voice control. With multiple HDMI 2.1 inputs, you can easily connect all your devices, making this a versatile and powerful addition to your entertainment setup.', price=998.0, url='https://www.dealnews.com/products/Hisense/Hisense-U7-Series-85-U7-K-85-4-K-144-Hz-ULED-UHD-Smart-TV-2023/459711.html?iref=rss-c142'), Deal(product_description="This Apple MagSafe Charger ensures your devices charge quickly and effortlessly with its 20W USB-C magnetic alignment technology. Designed specifically for iPhones and other compatible devices, the charger offers a seamless charging experience and easy attachment. Lightweight and compact, it's pe

## The Price is Right
First we'll polish off 2 more simple agents:

The Messaging Agent to send push notifications

The Planning Agent to coordinate activities

Then we'll put it all together into an Agent Framework.

For the Push Notification, we will be using a nifty platform called Pushover.
we'll need to set up a free account and add 2 tokens to our .env file:

PUSHOVER_USER=xxx
PUSHOVER_TOKEN=xxx

In [16]:
from dotenv import load_dotenv
from agents.messaging_agent import MessagingAgent

In [17]:
load_dotenv(override=True)
DB = "products_vectorstore"

In [18]:
# Create the MessagingAgent() instance and test by pushing a sample message (“MASSIVE NEWS!!!”).

agent = MessagingAgent()

In [19]:
# This should send me a push notification on my registered device
agent.push("MASSIVE NEWS!!!")

In [20]:
# The final step — connecting everything with a PlanningAgent, which coordinates between other agents (like messaging, pricing, and scanning)
import chromadb
DB = "products_vectorstore"
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')
from agents.planning_agent import PlanningAgent

In [21]:
planner = PlanningAgent(collection)

In [22]:
planner.plan()

Opportunity(deal=Deal(product_description='The Sumry 4,000W Hybrid Solar Inverter converts DC power from solar panels or batteries to AC power for home use. It features a built-in 140A MPPT solar charger, allowing it to efficiently charge 24V lead-acid and lithium batteries. This inverter is suitable for various applications, combining renewable energy technology with reliable performance, making it an ideal choice for energy independence.', price=244.0, url='https://www.dealnews.com/products/Sumry/Sumry-4-000-W-Hybrid-Solar-Inverter/490788.html?iref=rss-c142'), estimate=527.9321901899604, discount=283.93219018996035)